# **Example Aggregation Zones**

In some case it is useful to group objects in specific zones to facilitate analysis and/or visualization of the model result. This jupyter notebook demonstrates how to create a **spatial joint** of the FIAT model data with single or multiple **aggregation zones**. 
<p>A basic FIAT model can be set up by following the steps from the example notebook <a href="(https://github.com/Deltares/hydromt_fiat/blob/%2364FlexibleAggregationArea/examples/global_OSM_JRC.ipynb)"><em>global_OSM_JRC.ipynb</em></a>.

<em>Note: In this notebook the model was already initialized. Unless the user does not wish to use own data, there is `no need` to create a new model and the user can simply continue with the example data of this notebook.</p></em>

# **Step 0**: Create and activate the FIAT environment 
Using a <em>yaml</em> file, the neccessary environment is created and activated to run this notebook.
<p>---see this code </p>

# **Step 1**: Import required packages

Now we can import the neccessary python packages in our environment to build the **FIAT model**.

In [45]:
#Import the required python packages to run this notebook. 
import os
from typing import Sequence
from _pytest.mark.structures import ParameterSet
from hydromt_fiat.fiat import FiatModel
from hydromt.log import setuplog
from pathlib import Path
import pytest
import geopandas as gpd
import pandas as pd
from hydromt_fiat.fiat import workflows
import json
import hydromt_fiat
from hydromt_fiat import workflows
import hydromt
from hydromt import workflows
import yaml
#from hydromt_fiat.workflows.exposure_vector import ExposureVector
#from hydromt_fiat.workflows.vulnerability import Vulnerability
#from hydromt_fiat.workflows.aggregation_areas import join_exposure_aggregation_areas
#from hydromt_fiat.workflows.aggregation_areas import join_exposure_aggregation_multiple_areas

# **Step 2**: Configure and initialize the FIAT model

In this notebook the **FIAT model** was already initialized and the data is stored in the <em>"example"/ "data" / "example_aggregation_zone" </em> - directory. 
<p> The study area is located in Charelston, United States. Input data was obtained from **<** </p>

<em> Note: To configure and initialize your **own** FIAT model refer to  <a href="(https://github.com/Deltares/hydromt_fiat/blob/%2364FlexibleAggregationArea/examples/global_OSM_JRC.ipynb)"><em>global_OSM_JRC.ipynb</em></a>. If you do so, `do not` forget to update the folder dependencies and variables throughout this notebook according to your data. </em> 

# **Step 3**: Define aggregation parameters in the <em>yaml</em> file 

Two test cases are created in this jupyter notebook. First, the **spatial joint** of a **single** aggregation zone to the exposure data. Second, the **spatial joint** of **multiple** aggregation zones.

The test_case configuration is saved in a <em>yaml</em> file, in which the following parameters are specified:
- **new_root** (Directory path in which the new model output should be stored) 
- **aggregation_area_fn** (File path to the aggregation vector file) 
- **attribute_names** (name of the column that specifies the aggregation zone in the aggregation vector file(case-sensitive)) 
- **label_names** (new aggregation label name, chosen by the user)

<em>In this case below, the column `"ZONE_BASE"`categorizes the aggregation zones in the vector file. This column name is used for `attribute_names` . <em>

![attribute_name.png](img/Aggregation_zones.PNG)

## **Step 3.1:** Load the information of the <em>yaml</em> file

<p>The <em>yaml</em> file is located in the <em>"example"/ "data" /</em>  - folder. The information in the file are read and converted into a python dictionary. </p>
<p><em>Note: Instead of loading a yaml file it is possible to create a python dictionary directly, built upon the structure seen below.</em></p>

In [77]:
#Let's read the yaml file with the required information 
#and set up the FIAT model for the two test cases. 

with open(Path(os.path.abspath("")) / "data" / "aggregation_zones_example" / "config_aggregation.yml", 'r') as file:
    config_aggregation = yaml.safe_load(file)

print(json.dumps(config_aggregation, indent=4, sort_keys=False))

{
    "cases": {
        "test1_single_aggregation": {
            "new_root": "examples/data/Aggregation_zones_example/output/aggregation_zones_test1",
            "configuration": {
                "setup_aggregation_areas": {
                    "aggregation_area_fn": "examples/data/Aggregation_zones_example/Aggregation_zones/base_zones.gpkg",
                    "attribute_names": "ZONE_BASE",
                    "label_names": "Base_zones"
                }
            }
        },
        "test2_multiple_aggregation": {
            "new_root": "examples/data/Aggregation_zones_example/output/aggregation_zones_test2",
            "configuration": {
                "setup_aggregation_areas": {
                    "aggregation_area_fn": [
                        "examples/data/Aggregation_zones_example/Aggregation_zones/base_zones.gpkg",
                        "examples/data/Aggregation_zones_example/Aggregation_zones/land_use.gpkg",
                        "examples/data/Aggregatio

# **Step 4**: <strong>Define variables for the FIAT model</strong>

Set up the root path to the FIAT model and the logger settings to re-build your model:
- `root`: Directory path from where the neccessary data for the FIAT model is stored.  </p>
- `logger`: The logger variable defines the frequencies of log-entries during the initialisation of the model.</p>

In [83]:
# Set up Fiat Model
root = Path(os.path.abspath("")) / "data" / "aggregation_zones_example"
# Set up the logger
logger = setuplog("hydromt_fiat", log_level=10)

2023-10-02 12:27:00,250 - hydromt_fiat - log - INFO - HydroMT version: 0.8.0


# **Step 5: Run the FIAT model**

It's time to run the model with the new configuration of the aggregation zones. The output will be 
stored in the prior defined <em>`"new_root"`</em> - directory. The output will be a **new <em>exposure.csv</em>**, in which each object is assigned **spatially** to an aggregation zone. 

In [84]:
#Start running the new model with the new exposure output 
fm = FiatModel(root=root, mode="r", logger=logger)
fm.read()
fm.build(write=False, opt=config_aggregation["cases"]["test1_single_aggregation"]["configuration"])
fm.set_root(config_aggregation["cases"]["test1_single_aggregation"]["new_root"])
fm.write()

2023-10-02 12:27:02,746 - hydromt_fiat - log - DEBUG - Appending log messages to file c:\Users\rautenba\hydromt_fiat\examples\data\aggregation_zones_example\hydromt.log.
2023-10-02 12:27:02,746 - hydromt_fiat - model_api - INFO - Initializing fiat model from hydromt_fiat (v0.2.0).
2023-10-02 12:27:02,746 - hydromt_fiat - fiat - INFO - Reading model data from c:\Users\rautenba\hydromt_fiat\examples\data\aggregation_zones_example
2023-10-02 12:27:02,746 - hydromt_fiat - model_api - ERROR - Model config file not found at c:\Users\rautenba\hydromt_fiat\examples\data\aggregation_zones_example\fiat_configuration.ini
2023-10-02 12:27:02,746 - hydromt_fiat - model_api - ERROR - Model config file not found at c:\Users\rautenba\hydromt_fiat\examples\data\aggregation_zones_example\fiat_configuration.ini


TypeError: 'NoneType' object is not iterable

# **Done!**  
Your **FIAT model** created a **spatial joint** with your original objects and the aggregation zones. <p>Let's have a look at the output!</p>